In [1]:
from gensim.models import AuthorTopicModel

model = AuthorTopicModel.load('model.atmodel')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
for topic in model.show_topics(num_topics=100):
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print()

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Manuel Mireles pretende josé_manuel 

Words: fuertes_lluvias sexual unam estaciones conducir superior mireles José Man

In [3]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

Wall time: 1.14 s


In [4]:

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

scale = 0.01
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

In [6]:
from gensim.similarities import MatrixSimilarity


index = MatrixSimilarity(model[list(model.id2author.values())])



In [8]:

from gensim import matutils
import pandas as pd


author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    sims = get_sims(model.get_author_topics(name))


    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            

    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [9]:
get_table('caudillomx')

,Author,Score,Size
87,caudillomx,1.000000,8
97,esquivelgerardo,0.768232,23
17,E_Q_,0.745728,18
36,JohnMAckerman,0.737533,123
120,macariomx,0.728379,207
136,rivapa,0.718393,39
99,genarolozano,0.697500,103
88,diario24horas,0.695922,905
53,Pajaropolitico,0.692635,614
130,politicomx,0.690694,539
